# Makefile

# Criação de Makefiles para códigos Fortran


Aqui apresentaremos um pequena introdução à criação de makefiles para os projetos Fortran 90/95 em sistema operacional Linux. Ela também pode ser adaptado, com algumas pequenasmodificações, para outras linguagens de programação. 



## O que é Makefile?

O  `make` é muito conveniente para gerenciar códigos grandes ou grupos de códigos. Quando os códigos começam a ficar  cada vez maiores é visível a diferença de tempo necessário para  recompilar esses códigos em comparação com outros menores. Por outro lado, normalmente trabalha-se apenas em uma pequena parte do código (tal como uma simples função que se está fazendo uma depuração ou *debug*), e grande parte do resto do código permanece inalterada.

O `make` é uma ferramenta que controla a geração de executáveis, bibliotecas e outros arquivos não-fonte de um programa a partir dos arquivos fonte do programa. O `make` obtém seu conhecimento de como construir seu programa a partir de um arquivo,  o qual chama-se  `Makefile` ou `makefile`, que lista cada um dos arquivos que não são fontes e como obté-los a partir de outros arquivos. Ao escrever um programa, deve-se escrever um `Makefile` para ele, para que seja possível usar o  `make`  para construir e instalar o programa. O utilitário GNU `make` determina automaticamente quais partes de um programa grande precisam ser recompiladas e emite comandos para recompilá-las. 

Os exemplos apresentados mostram programas Fortran (F90/95), entretanto, pode-se usar o  `make` com qualquer outra linguagem de programação, como por exemplo o C ou C++, cujo compilador possa ser executado com um comando shell. De fato, o `make` não se limita aos programas. Pode-se usá-lo para descrever qualquer tarefa na qual alguns arquivos devem ser atualizados automaticamente assim que outros arquivos externos forem alterados. Para maiores detalhes consulte o [manual do make da GNU](https://www.gnu.org/software/make/manual/).

Para iniciar a usar o `make`, deve-se escrever um arquivo chamado "Makefile" ou "makefile" que descreva as relações entre os arquivos do seu programa e forneça comandos para atualizar cada arquivo. Em um programa, normalmente, o arquivo executável é atualizado a partir de arquivos de objeto, que são, por sua vez, compilados por arquivos de origem. Uma vez que há um "Makefile" adequado, cada vez que  se alterar qualquer um dos  arquivos fonte, o comando shell :

```bash
make
```

será suficiente para realizar todas as recompilações necessárias. Se houver vários arquivos makefile no mesmo diretório do projeto (o arquivo makefile padrão deve ser nomeado: Makefile), e alguns desses arquivos makefile têm nomes diferentes, então é pode executá-los especificando qual makefile deseja-se usar, com o seguinte comando:

```bash
make -f Nome_do_makefile
```

no qual `Nome_do_makefile` é o nome do makefile que se deseja executar.  Cada um desses makefiles pode ser completamente diferente um do outro. O programa `make` usa a base de dados makefile e os tempos de última modificação dos arquivos para decidir quais dos arquivos precisam ser atualizados. Para cada um desses arquivos, ele emite os comandos registrados no banco de dados.


## Introdução aos makefiles


É o arquivo  "Makefile" que diz `make` o que fazer para construir o programa executável. Na maioria das vezes, o makefile diz ao `make` como compilar e vincular um programa. Vamos discutir apenas um makefiles simples que descrevem como compilar e vincular um programa fortran que consiste em vários arquivos fonte Fortran 90/95. O makefile também pode dizer ao `make` como executar diversos comandos quando solicitado explicitamente (por exemplo, para remover certos arquivos como uma operação de limpeza). 

Considere que temos um código Fortran 90, com inúmeras subrotinas, módulos e funções, e que para esse código temos um Makefile o qual permite construir o programa executável desse código. Após uma otimização um desses arquivos fonte do código Fortran 90 foi alterado. Então, todos os arquivos do código fonte Fortran 90 precisam ser compilados e linkados para produzir o arquivo executável que executa o programa.

Quando o `make` recompila o programa, cada arquivo fonte do código Fortran 90 alterado deve ser recompilado também. Se um arquivo de módulo foi alterado (o qual mantém os parâmetros globais acessados por outros arquivos), cada arquivo fonte do código Fortran 90 que inclui este arquivo de módulo deve ser recompilado também. Cada compilação produz um arquivo de objeto correspondente ao arquivo de origem (com extensão: `.o`). Finalmente, se qualquer arquivo fonter tiver sido recompilado, todos os arquivos de objeto, sejam eles recém-criados ou obtidos de compilações anteriores, devem ser linkados para produzir o novo arquivo executável.

Um makefile simples consiste em "regras" com a seguinte forma:

```makefile
    ALVO ... : PRÉ-REQUISITOS ...
            COMANDO
            ...
```

O  "ALVO" é geralmente o nome de um arquivo que é gerado por um programa; exemplos de alvos são executáveis ou arquivos de objeto. Um alvo também pode ser o nome de uma ação a ser executada, como `clean`. Um "PRÉ-REQUISITO" é um arquivo usado como entrada para criar o destino. Um alvo geralmente depende de vários arquivos. Um "COMANDO" é uma ação que o `make` executa. Uma regra pode ter mais de um comando, cada um em sua própria linha.

**Observe:** É necessário colocar um caractere de tabulação no início de cada linha de comando! Esta é uma obscuridade que pega os desavisados.

Normalmente, um comando está em uma regra com pré-requisitos e serve para criar um arquivo de destino se qualquer um dos pré-requisitos for alterado. No entanto, a regra que especifica comandos para o destino não precisa ter pré-requisitos. Por exemplo, a regra que contém o comando delete associado ao alvo `clean` não possui pré-requisitos. Uma "regra", então, explica como e quando refazer certos arquivos que são os alvos da regra específica. `make` executa os comandos nos pré-requisitos para criar ou atualizar o alvo. Uma regra também pode explicar como e quando realizar uma ação.

Um makefile pode conter outro texto além das regras, mas um simples makefile precisa conter apenas regras. As regras podem parecer um pouco mais complicadas do que as mostradas neste tutorial, mas todas se encaixam no padrão mais ou menos.

## Um exemplo  simples 

Para demonstrar como escrever makefiles, vamos começar com um exemplo simples. Usaremos como compilador padrão o gfortran-8 e sua sintaxe.  Poderia-se usar um outro compilador qualquer, o que muda ria somente a sintaxe das opções de compilação do compilador em questão. Basicamente a sintaxe de compilação de todos os compiladores fortran não mudam muito. 

Inicialmente, será compilando e linkado (construído) um executável para um exemplo simples, que é descrito a seguir.

Como  exemplo usaremos um programa o qual irá realizar uma integração numérica, da seguinte função $f(x) = x \cos(x)$, a qual possui uma solução analítica

$$
\int_{a}^{b} x \cos(x) dx = \left. \left[ x \text{sen}(x) - \cos(x) \right]\right|_{a}^{b}
$$

Esssa função foi usada para poder validar os métodos de integração numérica.

Considere que este código é composto por quatro arquivos fonte fortran (todos eles terminam com extensão .f95), como segue:

```bash
principal.f95     -->  Realiza os testes de integração  
global.f95        -->  Um módulo com alguns parâmtros globais
integrais.f95     -->  Implementa os dois métodos de integração: Simpson 1/3 e trapezio
integrando.f95    -->  Define a função a ser integrada e o resultado exato.
```

O arquivo fonte: **principal.f95** contém o programa principal, o qual chama subsequentemente as outras funções. Cada programa  fortran deve ter um único programa principal com uma palavra chave: `PROGRAM`. Arquivo fonte: **global.f90** é um arquivo de módulo contendo diversas definições e/ou declarações de variáveis globais (ele também poderia incluir algumas funções e/ou sub-rotinas)). Este módulo é usado pelo programa principal e pelas demais funções definidas nos arquivos: **integrais.f95** e  **integrando.f95**. O arquivo **integrais.f95** possui as funções `simpson13` e `trapezio` enquanto o arquivo **integrando.f95** possui as funções `f1` e `f1r`. No programa principal, arquivo **principal.f95**, são realizadas chamadas diretas das funções `f1r`, `trapezio` e `simpson13`. 

A seguir vamos escrever esses arquivos para fique evidente seu conteudo.

In [39]:
#Primeiro vamos criar um diretório para gravar esses arquivos
! mkdir -p exemplo_make

In [40]:
%%writefile exemplo_make/global.f95
MODULE PRECISAO
  IMPLICIT NONE
  ! Setting the kind of REAL's
  INTEGER, PARAMETER :: sp = SELECTED_REAL_KIND(5,30)    ! precisao simple
  INTEGER, PARAMETER :: dp = SELECTED_REAL_KIND(10,100)  ! precisao dupla
  INTEGER, PARAMETER :: qp = SELECTED_REAL_KIND(18,4000) ! precisao quadupla
  INTEGER, PARAMETER :: op = SELECTED_REAL_KIND(25,4000) ! precisao octupla
  ! Setting the selected REAL kind
  INTEGER, PARAMETER :: pr = dp
  ! Setting the kind of INTEGER's
  INTEGER, PARAMETER :: ip1 = selected_int_KIND(2)
  INTEGER, PARAMETER :: ip2 = selected_int_KIND(4)
  INTEGER, PARAMETER :: ip3 = selected_int_KIND(8)
  INTEGER, PARAMETER :: ip4 = selected_int_KIND(10)
  INTEGER, PARAMETER :: ip5 = selected_int_KIND(20)
  !---------------------------------------------------------------------
  !  Frequently used mathematical constants (with precision to spare):
  !--------------------------------------------------------------------- 
  REAL(SP), PARAMETER :: PI       = 3.141592653589793238462643383279502884197_sp
  REAL(SP), PARAMETER :: PIO2     = 1.57079632679489661923132169163975144209858_sp
  REAL(SP), PARAMETER :: TWOPI    = 6.283185307179586476925286766559005768394_sp 
  REAL(SP), PARAMETER :: SQRT2    = 1.41421356237309504880168872420969807856967_sp
  REAL(SP), PARAMETER :: EULER    = 0.5772156649015328606065120900824024310422_sp
  REAL(DP), PARAMETER :: PI_DP    = 3.141592653589793238462643383279502884197_dp
  REAL(DP), PARAMETER :: PIO2_DP  = 1.57079632679489661923132169163975144209858_dp
  REAL(DP), PARAMETER :: TWOPI_DP = 6.283185307179586476925286766559005768394_dp
  REAL(DP), PARAMETER :: ONEO2PI    = 0.15915494_dp
  !--------------------------------------------------------------------- 
  REAL(DP), PARAMETER :: EIGHT_PI = 4.0_DP * TWOPI_DP
  !--------------------------------------------------------------------- 
  !  Frequently used physical constants (with precision to spare):
  !--------------------------------------------------------------------- 
  REAL(PR), PARAMETER :: KB  = 8.61738573E-2_PR      ! Cte. de Boltzman em meV/K
  REAL(PR), PARAMETER :: A0  = 0.52917724924_PR      ! Raio de Bohr em Angstrons
  REAL(PR), PARAMETER :: RY  = 13.60569809E+3_PR     ! Rydberg em meV
  REAL(PR), PARAMETER :: MB  = 0.057883826352_PR     ! Magneton de Bohr em meV/T
  REAL(PR), PARAMETER :: GM  = 2.00231930438620_PR   ! Fator giromagnetico do eletron
  REAL(PR), PARAMETER :: HB  = 658.21122020_PR       ! Cte. de Planck em mev*fsec
  REAL(PR), PARAMETER :: HBps= 0.65821122020_PR      ! Cte. de Planck em mev*psec
  REAL(PR), PARAMETER :: HBAR = 6.58221122020E-13_PR ! Em meV*s
  REAL(PR), PARAMETER :: HR  = 20.67071735_PR        ! (HBAR/Ry) em fsec^(-1) 
  REAL(PR), PARAMETER :: HWC = 0.1157676524_PR       ! (HBAR * W_C) em meV/T 
  REAL(PR), PARAMETER :: GWC = 8.508762422E-6_PR     ! (HBAR * W_C)/Ry em 1/T
  REAL(PR), PARAMETER :: QE  = 1.60217733497E-19_PR  ! Em Coulombs
END MODULE PRECISAO

Overwriting exemplo_make/global.f95


In [41]:
%%writefile exemplo_make/integrais.f95
!====================================================================
FUNCTION simpson13(a, b, n, func)
  USE precisao
  !-----------------------------------------------------------------
  IMPLICIT NONE
  !-----------------------------------------------------------------
  INTERFACE
     FUNCTION func(x)
       USE precisao
       REAL(pr), INTENT (IN) :: x
       REAL(pr)              :: func
     END FUNCTION func
  END INTERFACE
  !-----------------------------------------------------------------
  REAL(pr), INTENT(in) :: a, b  ! Limites de integracao 
  INTEGER,  INTENT(in) :: n     ! Numero de ptos. 
  REAL(pr)             :: simpson13    ! Decaclara o tipo da funcao
  !-----------------------------------------------------------------
  INTEGER  :: i, m
  REAL(pr) :: dx, s0, soma, x
  !-----------------------------------------------------------------
  !     O erro eh da ordem de (dx**5 /90)*f''''(x)
  !-----------------------------------------------------------------
  !     O numero de pontos deve ser do tipo n=2m+1, um multiplo de 2 mais um
  !-----------------------------------------------------------------
  soma = 0.0_pr
  dx = (b - a ) / (n - 1)
  s0 = FUNC(a) - FUNC(b)
  !-----------------------------------------------------------------
  m = ( n - 1) /2
  !-----------------------------------------------------------------
  DO i= 1, m
     x  = a + (2*i-1)*dx
     soma = soma + 2.0_pr * func(x) + func(x+dx)
  END DO
  !-----------------------------------------------------------------
  simpson13 = dx * ( (s0 + 2.0_pr * soma ) / 3.0_pr)
  !-----------------------------------------------------------------
  RETURN
  !-----------------------------------------------------------------
END FUNCTION simpson13
!====================================================================


!====================================================================
FUNCTION trapezio (a, b, n, func)
  USE precisao
  !-----------------------------------------------------------------
  IMPLICIT NONE
  !-----------------------------------------------------------------
  INTERFACE
     FUNCTION func(x)
       USE precisao
       REAL(pr), INTENT (in) :: x
       REAL(pr)              :: func
     END FUNCTION func
  END INTERFACE
  !-----------------------------------------------------------------
  REAL(pr) :: trapezio
  REAL(pr) :: a, b, dx, x, soma
  INTEGER  :: i, n
  !-----------------------------------------------------------------
  dx = (b - a) / REAL(n)
  soma = 0.0_pr
  !-----------------------------------------------------------------
  DO i = 1, n-1
     x  = a + i * dx       ! evita-se erros acumulativos
     soma = soma + func(x)
  END DO
  !-----------------------------------------------------------------
  trapezio = (2.0_pr * soma + func(a) + func(b)) * dx / 2.0_pr
  !-----------------------------------------------------------------
  RETURN
  !-----------------------------------------------------------------
END FUNCTION trapezio
!====================================================================

Overwriting exemplo_make/integrais.f95


In [42]:
%%writefile exemplo_make/integrando.f95
FUNCTION f1(x)
  USE PRECISAO
  IMPLICIT NONE
  REAL(pr), INTENT (IN) :: x
  REAL(pr)              :: f1
  f1 = x * COS(x)
END FUNCTION f1

FUNCTION f1r(x)
  USE PRECISAO
  IMPLICIT NONE
  REAL(pr), INTENT (IN) :: x
  REAL(pr)              :: f1r
  f1r = x * SIN(x) + COS(x)
END FUNCTION f1r

Overwriting exemplo_make/integrando.f95


In [43]:
%%writefile exemplo_make/principal.f95
!=======================================================================
PROGRAM Testa_Integrais
  USE precisao
  IMPLICIT NONE
  REAL (pr) :: x0, xf , exato, erro, r
  INTEGER   :: n, np
  !---------------------------------------------------------------------
  INTERFACE
     FUNCTION f1r(x)
       USE precisao
       REAL(pr), INTENT (IN) :: x
       REAL(pr)              :: f1r
     END FUNCTION f1r

     FUNCTION f1(x)
       USE precisao
       REAL(pr), INTENT (IN) :: x
       REAL(pr)              :: f1
     END FUNCTION f1

     FUNCTION simpson13(a, b, n, func)
       USE precisao
       REAL(pr), INTENT(in) :: a, b         ! Limites de integracao 
       INTEGER,  INTENT(in) :: n            ! Numero de ptos. 
       REAL(pr)             :: simpson13    ! Decaclara o tipo da funcao
       INTERFACE
          FUNCTION func(x)
            USE precisao
            REAL(pr), INTENT (IN) :: x
            REAL(pr)              :: func
          END FUNCTION func
       END INTERFACE
     END FUNCTION simpson13

     FUNCTION trapezio(a, b, n, func)
       USE precisao
       REAL(pr), INTENT(in) :: a, b         ! Limites de integracao 
       INTEGER,  INTENT(in) :: n            ! Numero de ptos. 
       REAL(pr)             :: trapezio     ! Decaclara o tipo da funcao
       INTERFACE
          FUNCTION func(x)
            USE precisao
            REAL(pr), INTENT (IN) :: x
            REAL(pr)              :: func
          END FUNCTION func
       END INTERFACE
     END FUNCTION trapezio
  END INTERFACE
  !---------------------------------------------------------------------
  WRITE(*,*) 'Entre com os limites de integracao'
  READ(*,*) x0, xf
  WRITE(*,*) 'Entre com o numero ptos.'
  READ(*,*)  np
  !----------------------------------------------------------------------
  exato = f1r(xf)-f1r(x0)
  WRITE(*,*) ' Exato          : ', exato 
  ! ----------------------------------------------------------------------
  n = ((np-1)/2) 
  n = 2 * n + 1
  r    = simpson13(x0, xf, n, f1) ! Integral por Simpson13
  erro = ABS( exato - r )
  WRITE(*,*) ' Simpson13     : ', r, ' Erro = ', erro
  ! ----------------------------------------------------------------------
  r    = trapezio(x0, xf, np, f1) ! Integral por Trapezio
  erro = ABS( exato - r )
  WRITE(*,*) ' Trapezio       : ', r, ' Erro = ', erro
  ! ----------------------------------------------------------------------
  WRITE(*,*)
END PROGRAM Testa_Integrais
!=======================================================================

Overwriting exemplo_make/principal.f95


Para construir o programa principal, é necessário compilar todos os quatro arquivos de origem e linká-los para produzir o arquivo executável. Usando o gfortran, essa tarefa pode ser realizada com o seguinte comando (no shell):

```bash
gfortran -Wall -std=f2008 global.f95 principal.f95 integrais.f95 integrando.f95
```

que deve produzir um arquivo chamado: `a.out`. Este é um arquivo executável para este programa. O compilador compilou todos os quatro arquivos de origem e produziu os respectivos arquivos de objeto (`.o`) e os linkou/vinculou automaticamente. No diretório atual, deve-se encontrar os seguintes arquivos adicionais produzidos pelo compilador: `global.o`, `global.mod`, `principal.o`, `integrais.o` e `integrando.o`. Arquivos terminados em `.o` são arquivos de objeto, enquanto `global.mod` é um arquivo de módulo. Este arquivo usa o nome do módulo (aqui, o módulo e o arquivo de origem possuem o mesmo nome: global). Isso nem sempre é o caso, mas o princípio continua o mesmo.

In [44]:
# Vamos compilar o código a seguir 
! f95 -std=f2008 exemplo_make/global.f95 exemplo_make/integrando.f95 \
               exemplo_make/integrais.f95  exemplo_make/principal.f95

In [45]:
#Vejamos o resultado gue foram gerados na pasta raiz
! ls -alh *.o *.mod a.out

ls: não foi possível acessar '*.o': Arquivo ou diretório inexistente
-rwxr-xr-x 1 salviano salviano  14K fev  2 09:22  a.out
-rw-r--r-- 1 salviano salviano 1,1K fev  2 09:19  precisao.mod


In [46]:
# Rodando o programa
! echo '1 4\n 100\n' | ./a.out

 Entre com os limites de integracao
 Entre com o numero ptos.
  Exato          :   -5.0626268927713607     
  Simpson13     :   -5.0626268941696519       Erro =    1.3982912605570164E-009
  Trapezio       :   -5.0624262873207657       Erro =    2.0060545059497059E-004



In [47]:
# Antes de continuar vamos remofer o lixo gerado
! rm -f a.out precisao.mod

Se desejamo um arquivo executável com um nome mais significativo que `a.out`, deve-se usar a opção  `-o` que é padrão para *output*, e nesse caso o comando anteriro toma a seguinte forma:

```bash
gfortran -Wall -std=f2008 -o test global.f95 integrando.f95 integrais.f95 principal.f95
```

que deve produzir um arquivo chamado: `test`. Este é um arquivo executável, que pode ser executado no shell com o seguinte comando:

```bash
./test
```

Para compilar um único arquivo fonte sem vinculação automática, use  o seguinte comando:

```bash
gfortran -Wall -std=f2008 -c integrando.f95
```

que deve produzir um arquivo chamado: `integrando.o`. Este é um arquivo de objeto para o arquivo fonte: `integrando.f95`. Cada arquivo fonte tem um arquivo de objeto correspondente, que é produzido como um produto de um processo de compilação e é usado no processo de vinculação. Para compilar um arquivo fonte que referencia (usa) o arquivo de módulo (através da instrução USE) é necessário compilar o arquivo de módulo primeiro. Por exemplo:

```bash
gfortran -Wall -std=f2008 -c global.f95
gfortran -Wall -std=f2008 -c principal.f95
```

In [48]:
#Vamos compilar 
! f95 -Wall -std=f2008 -c exemplo_make/global.f95
! f95 -Wall -std=f2008 -c exemplo_make/integrais.f95
! f95 -Wall -std=f2008 -c exemplo_make/integrando.f95
! f95 -Wall -std=f2008 -c exemplo_make/principal.f95

In [49]:
# Vamos verificar os arquivos gerados 
! ls -alh *.o *.mod

-rw-r--r-- 1 salviano salviano  944 fev  2 09:22 global.o
-rw-r--r-- 1 salviano salviano 2,2K fev  2 09:22 integrais.o
-rw-r--r-- 1 salviano salviano 1,7K fev  2 09:22 integrando.o
-rw-r--r-- 1 salviano salviano 1,1K fev  2 09:22 precisao.mod
-rw-r--r-- 1 salviano salviano 5,0K fev  2 09:22 principal.o


O primeiro comando compila o `global.f95` (sem vincular) e produz dois arquivos: `global.o`, que é um arquivo de objeto, e o `precisao.mod` (aqui, o módulo é denominado `precisao`). Este arquivo posterior é um arquivo de módulo, e é necessário no processo de compilação (segundo comando) do `principal.f95` (já que está usando este módulo através da instrução `USE`), e precisa estar disponível no momento da compilação. Portanto, a ordem dos comandos acima não deve ser alterada (reordenada)!

Ligação final de todos os arquivos compilados (arquivos de módulo) é realizada com o seguinte comando:

```bash
gfortran global.o integrais.o integrando.o principal.o
```

In [50]:
# Note que todos foram gerados na pasta raiz, logo
! f95 global.o integrais.o integrando.o principal.o

In [51]:
#Ele certamente gerou um exscutávael a.out
! ls -alh a.out

-rwxr-xr-x 1 salviano salviano 14K fev  2 09:22 a.out


In [52]:
# Vamos rodar ele
! echo '1 4\n 100\n' | ./a.out

 Entre com os limites de integracao
 Entre com o numero ptos.
  Exato          :   -5.0626268927713607     
  Simpson13     :   -5.0626268941696519       Erro =    1.3982912605570164E-009
  Trapezio       :   -5.0624262873207657       Erro =    2.0060545059497059E-004



qual deve produzir o arquivo: `a.out`, que é executável. Para atribuir um nome diferente de `a.out` ao executável gerado durante o processo de vinculação, deve-se usar o seguinte comando (por exemplo):

```bash
gfortran -o test global.o integrais.o integrando.o principal.o
```

ou ainda 

```bash
gfortran -o test *.o

```

o que deve produzir o arquivo executável chamado: `test`. O asterisco no último comando é um símbolo curinga que significa: inclua todos os arquivos `.o` no diretório atual. Símbolos curinga podem ser usados ao emitir comandos do compilador e/ou vinculador.

**Observe:**
Para usar um compilador fortran diferente, como o g95, por exemplo, ele deve apenas trocar gfortran com g95 em todos os comandos acima. Tudo que foir dito anteriormente para o gfortran vale também para o g95 e para o Intel Fortran Compiler. Todos os exemplos acima podem ser executados com este compilador, apenas troque o gfortran pelo g95 ou o ifort da Intel Fortran Compiler.

Ao construir programas fortran (que consistem em vários arquivos de código fonte, incluindo um ou mais arquivos de módulo - todos localizados no mesmo diretório (funcional)) da linha de comando no shell (bash), o procedimento a seguir é a maneira mais rápida de produzir arquivo executável. Primeiro compilar sem vincular todos os arquivos de origem contendo a declaração do módulo. Em seguida, emita o seguinte comando: `gfortran -Wall -std=f2008 -o test *.f95` que compila (sem vincular) todos os arquivos de origem F90/95 encontrados no diretório atual (em funcionamento). Este comando produz arquivos de objetos necessários (arquivos `.o`) para cada arquivo de código fonte (arquivo` .f95`). Depois disso, o seguinte comando para ligar todos os arquivos objeto: `gfortran -o test *.o` que produz um arquivo executável, chamado: `test`.

Toda vez que um dos arquivos de origem é alterado, é necessário compilá-lo e vinculá-lo a todos os outros arquivos de objeto necessários para produzir o arquivo executável. Se o número de arquivos de origem for grande, isso pode ser difícil. Aí vem o utilitário makefile para facilitar esse processo.

In [53]:
# Note que todos foram gerados na pasta raiz, logo
! f95 -o test global.o integrais.o integrando.o principal.o

In [54]:
# Vamos rodar ele
! echo '1 4\n 100\n' | ./test

 Entre com os limites de integracao
 Entre com o numero ptos.
  Exato          :   -5.0626268927713607     
  Simpson13     :   -5.0626268941696519       Erro =    1.3982912605570164E-009
  Trapezio       :   -5.0624262873207657       Erro =    2.0060545059497059E-004



In [55]:
# Antes de continuar vamos remofer o lixo gerado
! rm -f a.out precisao.mod test

## Um arquivo Makefile exemplo

O mesmo processo de compilar e vincular arquivos de origem realizado no exemplo acima com o uso da linha de comando do shell para compilar e vincular pode ser facilmente realizado com o auxilio de um arquivo Makefile. Portanto, é necessário escrever um makefile apenas uma vez e construir o programa (compilar + linkar/vincular = executável) com um comando: **make** na linha de comando do shell.

Aqui está um makefile simples que descreve a maneira como um arquivo executável é chamado: test depende de quatro arquivos de origem (um dos quais é um arquivo de módulo). Este makefile é escrito para o exemplo apresentado acima.

```makefile
# Esta é uma linha de comentário em um makefile
# Início do  makefile

test: global.o integrais.o integrando.o principal.o
	gfortran -o test global.o integrais.o integrando.o principal.o

precisao.mod: global.o global.f95
	gfortran -Wall -std=f2008 -c global.f95

global.o: global.f95
	gfortran -Wall -std=f2008 -c global.f95

principal.o: precisao.mod principal.f95
	gfortran -Wall -std=f2008 -c principal.f95

integrando.o: precisao.mod integrando.f95
	gfortran -Wall -std=f2008 -c integrando.f95

integrais.o: precisao.mod integrais.f95
	gfortran -Wall -std=f2008 -c integrais.f95

clean:
	rm precisao.mod global.o principal.o integrando.o integrais.o test

# Fim do makefile
```

Este arquivo deve ser salvo no diretório onde estão os arquivos fonte e deve receber o seguinte nome: **Makefile**. Longas linhas em um makefile poderiam ser continuadas com o seguinte símbolo: `\`. Linhas de comentários são pressionadas com o seguinte símbolo: `#`.

Para usar este makefile para criar o arquivo executável chamado `test`, digite:

```bash
make

```

Para usar este makefile para excluir o arquivo executável e todos os arquivos de objeto do diretório, digite:

```bash
make clean

```

Quando um "alvo" é um arquivo, ele precisa ser recompilado ou vinculado novamente se algum de seus pré-requisitos for alterado. Além disso, quaisquer pré-requisitos que sejam gerados automaticamente devem ser atualizados primeiro. Um comando shell segue cada linha que contém um destino e pré-requisitos. Esses comandos shell dizem como atualizar o arquivo de destino. Um caractere de tabulação deve vir no início de cada linha de comando para distinguir linhas de comandos de outras linhas no makefile. (Tenha em mente que o `make` não sabe nada sobre como os comandos funcionam. Cabe ao desenvolvedor/programador fornecer comandos que atualizarão o arquivo de destino adequadamente. Tudo o que `make` faz é executar os comandos na regra que específica quando o arquivo de destino precisa ser atualizado.)

O alvo `clean` não é um arquivo, mas apenas o nome de uma ação. Geralmente não se deseja executar as ações nesta regra, `clean` e ela não é um pré-requisito de qualquer outra regra. Consequentemente, o `make` nunca faz nada a menos que se diga especificamente. Observe que essa regra não apenas não é um pré-requisito, mas também não possui pré-requisitos, portanto, a única finalidade da regra é executar os comandos especificados. Destinos que não se referem a arquivos, mas são apenas ações, são chamados de "alvos falsos" do inglês "phony targets".

Por padrão, o `make` começa com o primeiro alvo. Isso é chamado de "objetivo padrão". No exemplo simples da seção anterior, o objetivo padrão é atualizar o programa executável `test`; portanto, colocamos essa regra primeiro. Assim, quando se dá um comando: `make`, ele lê o arquivo Makefile no diretório atual e começa processando a primeira regra. No exemplo, esta regra é para relink ou revincular; mas antes que o `make` possa processar totalmente esta regra, ele deve processar as regras para os arquivos que ela depende, que neste caso são os arquivos de objeto. Cada um desses arquivos é processado de acordo com sua própria regra. Estas regras dizem para atualizar cada arquivo `.o` compilando seu arquivo fonte. A recompilação deve ser feita se o arquivo fonte, ou qualquer um dos arquivos de cabeçalho chamados como pré-requisitos, for mais recente que o arquivo de objeto ou se o arquivo de objeto não existir. As outras regras são processadas porque seus destinos aparecem como pré-requisitos do objetivo. Se alguma outra regra não depender do objetivo (ou de qualquer coisa da qual ela dependa, etc.), essa regra não será processada, a menos que se diga `make` para fazer isso (com um comando como `make clean`).

Depois de recompilar qualquer objeto necessário, o `make` decide se irá revincular `test`. Isto deve ser feito se o arquivo `test` não existir, ou se algum dos arquivos objeto for mais recente que ele. Se um arquivo objeto foi apenas recompilado, ele é agora mais novo que `test`, então `test` é revinculado.

Aqui estão algumas regras úteis para se criar makefiles simples:

-  Todos os comandos devem ser precedidos por um caractere de tabulação.
-  A primeira linha no makefile deve definir o programa executável, listando todos os pré-requisitos (arquivos de objeto e módulo necessários), seguidos por um comando do compilador na próxima linha. 
-  O caractere de tabulação deve ser colocado na frente do comando do compilador! Depois de definir o destino (arquivo executável), deve-se seguir as definições para o arquivo de módulo (arquivo `.mod`) e, em seguida, o arquivo de objeto do arquivo de módulo correspondente (`.o` do arquivo de módulo). 
-  Para o arquivo `.mod`, os pré-requisitos correspondem aos arquivos `.o` e `.f95`. 
-  O caractere de tabulação deve ser colocado na frente do comando do compilador! Agora pode-se listar todos os outros arquivos de objetos (arquivos `.o`) com seus pré-requisitos, em qualquer ordem, seguidos por uma diretiva de compilador para construir esse arquivo de objeto (precedido por um caractere de tabulação). 
-  A ordem dos arquivos de objeto 'outros' em um makefile é completamente arbitrária. Ao listar um arquivo de objeto que usa módulo (como: `integrando.o`), o arquivo de módulo correspondente (arquivo `.mod`) deve ser especificado como pré-requisito!

A regra `clean` é um recurso arbitrário, mas útil, que deve ser implementado em todos os makefiles. Ele simplesmente exclui (limpa) todos os arquivos de objetos (e módulos) do diretório atual (funcionando), incluindo (se desejar) o arquivo executável.

In [56]:
%%writefile exemplo_make/Makefile
# Esta é uma linha de comentário em um makefile
# Início do  makefile

test: global.o integrais.o integrando.o principal.o
	gfortran -o test global.o integrais.o integrando.o principal.o

precisao.mod: global.o global.f95
	gfortran -Wall -std=f2008 -c global.f95

global.o: global.f95
	gfortran -Wall -std=f2008 -c global.f95

principal.o: precisao.mod principal.f95
	gfortran -Wall -std=f2008 -c principal.f95

integrando.o: precisao.mod integrando.f95
	gfortran -Wall -std=f2008 -c integrando.f95

integrais.o: precisao.mod integrais.f95
	gfortran -Wall -std=f2008 -c integrais.f95

clean:
	rm precisao.mod global.o principal.o integrando.o integrais.o test

# Fim do makefile

Overwriting exemplo_make/Makefile


In [57]:
# Vamos executar o make
! cd exemplo_make && make

gfortran -Wall -std=f2008 -c global.f95
gfortran -Wall -std=f2008 -c global.f95
gfortran -Wall -std=f2008 -c integrais.f95
gfortran -Wall -std=f2008 -c integrando.f95
gfortran -Wall -std=f2008 -c principal.f95
gfortran -o test global.o integrais.o integrando.o principal.o


In [58]:
# Vamos observar o resultado
! ls -alh exemplo_make/*

-rw-r--r-- 1 salviano salviano 2,9K fev  2 09:22 exemplo_make/global.f95
-rw-r--r-- 1 salviano salviano  944 fev  2 09:22 exemplo_make/global.o
-rw-r--r-- 1 salviano salviano 3,0K fev  2 09:22 exemplo_make/integrais.f95
-rw-r--r-- 1 salviano salviano 2,2K fev  2 09:22 exemplo_make/integrais.o
-rw-r--r-- 1 salviano salviano  292 fev  2 09:22 exemplo_make/integrando.f95
-rw-r--r-- 1 salviano salviano 1,7K fev  2 09:22 exemplo_make/integrando.o
-rw-r--r-- 1 salviano salviano  682 fev  2 09:22 exemplo_make/Makefile
-rw-r--r-- 1 salviano salviano  670 fev  1 10:59 exemplo_make/makefile1
-rw-r--r-- 1 salviano salviano  750 fev  1 13:14 exemplo_make/makefile2
-rw-r--r-- 1 salviano salviano  678 fev  2 08:12 exemplo_make/makefile3
-rw-r--r-- 1 salviano salviano  845 fev  2 09:16 exemplo_make/makefile4
-rw-r--r-- 1 salviano salviano 1,1K fev  2 09:22 exemplo_make/precisao.mod
-rw-r--r-- 1 salviano salviano 2,5K fev  2 09:22 exemplo_make/principal.f95
-rw-r--r-- 1 salviano salviano 5,0K fev  2 0

In [59]:
# Vamos rodar ele
! echo '1 4\n 100\n' | ./exemplo_make/test

 Entre com os limites de integracao
 Entre com o numero ptos.
  Exato          :   -5.0626268927713607     
  Simpson13     :   -5.0626268941696519       Erro =    1.3982912605570164E-009
  Trapezio       :   -5.0624262873207657       Erro =    2.0060545059497059E-004



In [60]:
# Agora vamos limpar 
! cd exemplo_make && make clean
! ls -alh exemplo_make/*

rm precisao.mod global.o principal.o integrando.o integrais.o test
-rw-r--r-- 1 salviano salviano 2,9K fev  2 09:22 exemplo_make/global.f95
-rw-r--r-- 1 salviano salviano 3,0K fev  2 09:22 exemplo_make/integrais.f95
-rw-r--r-- 1 salviano salviano  292 fev  2 09:22 exemplo_make/integrando.f95
-rw-r--r-- 1 salviano salviano  682 fev  2 09:22 exemplo_make/Makefile
-rw-r--r-- 1 salviano salviano  670 fev  1 10:59 exemplo_make/makefile1
-rw-r--r-- 1 salviano salviano  750 fev  1 13:14 exemplo_make/makefile2
-rw-r--r-- 1 salviano salviano  678 fev  2 08:12 exemplo_make/makefile3
-rw-r--r-- 1 salviano salviano  845 fev  2 09:16 exemplo_make/makefile4
-rw-r--r-- 1 salviano salviano 2,5K fev  2 09:22 exemplo_make/principal.f95


## Variáveis

Para tornar o processo de escrever makefiles mais agil e mais legível, é possível usar variáveis. Variáveis também são chamadas de macros. Por exemplo, no exemplo de makefile simples acima, a regra para tornar alvo: `test` lista todos os arquivos de objeto como pré-requisitos. Se vários arquivos no projeto forem grandes, essa lista poderá ficar bastante longa. Também pode ser visto no exemplo acima que a regra: `clean` também lista todos esses arquivos de objetos novamente.

Para reduzir isso, uma variável poderia ser definida da seguinte forma:

```makefile
objects = global.o integrais.o integrando.o principal.o

```

e, em seguida, use essa variável quando apropriado, da seguinte maneira:

```makefile
test: $(objects)
        gfortran -Wall -std=f2008 -o test $(objects)

clean:
        rm test $(objects)

```

Outra maneira útil de usar variáveis (macros) é definir o tipo/nome do compilador que é usado no makefile. Por exemplo, para usar o makefile acima com um compilador fortran g95, bastaria substituir cada comando 'gfortran' por um comando 'g95', em todo o makefile. Isso pode ser bastante difícil e sujeito a erros. A maneira mais fácil é definir o comando do compilador (variável) no início do makefile, como:

```makefile
F90 = gfortran
```

O mesmo makefile apresentado acima deve agora ser algo como isto:

```makefile
# Esta é uma linha de comentário em um makefile
# Início do  makefile

objects = global.o integrais.o integrando.o principal.o
F90 = gfortran-8

# Makefile
test: $(objects)
        $(F90) -o test $(objects)


precisao.mod: global.o global.f95
	$(F90) -Wall -std=f2008 -c global.f95

global.o: global.f95
	$(F90) -Wall -std=f2008 -c global.f95

principal.o: precisao.mod principal.f95
	$(F90) -Wall -std=f2008 -c principal.f95

integrando.o: precisao.mod integrando.f95
	$(F90) -Wall -std=f2008 -c integrando.f95

integrais.o: precisao.mod integrais.f95
	$(F90) -Wall -std=f2008 -c integrais.f95

# Limpando tudo
clean:
	rm precisao.mod test
	rm $(objects)

# Fim do makefile
```

Como exemplo vamos gerar esse makefile e vamos chamá-lo de `makefile1`:

In [61]:
%%writefile exemplo_make/makefile1
# Esta é uma linha de comentário em um makefile
# Início do  makefile

objects = global.o integrais.o integrando.o principal.o
F90 = gfortran-8

# Makefile
test: $(objects)
	$(F90) -o test $(objects)

precisao.mod: global.o global.f95
	$(F90) -Wall -std=f2008 -c global.f95

global.o: global.f95
	$(F90) -Wall -std=f2008 -c global.f95

principal.o: precisao.mod principal.f95
	$(F90) -Wall -std=f2008 -c principal.f95

integrando.o: precisao.mod integrando.f95
	$(F90) -Wall -std=f2008 -c integrando.f95

integrais.o: precisao.mod integrais.f95
	$(F90) -Wall -std=f2008 -c integrais.f95

# Limpando tudo
clean:
	rm precisao.mod test
	rm $(objects)

# Fim do makefile

Overwriting exemplo_make/makefile1


In [62]:
# Vamos executar o make
! cd exemplo_make && make -f makefile1

gfortran-8 -Wall -std=f2008 -c global.f95
gfortran-8 -Wall -std=f2008 -c global.f95
gfortran-8 -Wall -std=f2008 -c integrais.f95
gfortran-8 -Wall -std=f2008 -c integrando.f95
gfortran-8 -Wall -std=f2008 -c principal.f95
gfortran-8 -o test global.o integrais.o integrando.o principal.o


In [63]:
# Vamos rodar ele
! echo '1 4\n 100\n' | ./exemplo_make/test

 Entre com os limites de integracao
 Entre com o numero ptos.
  Exato          :   -5.0626268927713607     
  Simpson13     :   -5.0626268941696519       Erro =    1.3982912605570164E-009
  Trapezio       :   -5.0624262873207657       Erro =    2.0060545059497059E-004



In [64]:
# Agora vamos limpar 
! cd exemplo_make && make -f makefile1 clean
! ls -alh exemplo_make/*

rm precisao.mod test
rm global.o integrais.o integrando.o principal.o
-rw-r--r-- 1 salviano salviano 2,9K fev  2 09:22 exemplo_make/global.f95
-rw-r--r-- 1 salviano salviano 3,0K fev  2 09:22 exemplo_make/integrais.f95
-rw-r--r-- 1 salviano salviano  292 fev  2 09:22 exemplo_make/integrando.f95
-rw-r--r-- 1 salviano salviano  682 fev  2 09:22 exemplo_make/Makefile
-rw-r--r-- 1 salviano salviano  670 fev  2 09:22 exemplo_make/makefile1
-rw-r--r-- 1 salviano salviano  750 fev  1 13:14 exemplo_make/makefile2
-rw-r--r-- 1 salviano salviano  678 fev  2 08:12 exemplo_make/makefile3
-rw-r--r-- 1 salviano salviano  845 fev  2 09:16 exemplo_make/makefile4
-rw-r--r-- 1 salviano salviano 2,5K fev  2 09:22 exemplo_make/principal.f95


Agora, para alterar o comando do compilador de gfortran para, por exemplo, g95 ou ifort, tudo que se precisa fazer é alterar a linha onde o comando do compilador é definido (sob variáveis), como segue:

```makefile
F90 = g95
```

e é isso! Este mesmo makefile pode agora ser usado com um compilador fortran g95, com o comando 'make'. Variáveis são definidas no início do makefile! A variável útil que pode ser definida para os makefiles é aquela que define as opções do compilador. Desta forma, várias opções de comutador (`F90FLAGS`) poderiam ser ativadas/desativadas de uma maneira simples. Por exemplo, pode-se definir a seguinte variável:

```makefile
F90FLAGS = -Wall -std=f2008 -O3
```

que representa uma opção de otimização do compilador para os compiladores gfortran, g95 e  Intel Fortran Compiler no Linux. Com essa nova variável, o makefile acima apresentado se parece com isto:

```makefile
# Esta é uma linha de comentário em um makefile
# Início do  makefile
objects = global.o integrais.o integrando.o principal.o
F90 = gfortran-8
F90FLAGS = -Wall -std=f2008 -O3

# Makefile
test: $(objects)
	$(F90) -o test $(objects)

precisao.mod: global.o global.f95
	$(F90) $(F90FLAGS) -c global.f95

global.o: global.f95
	$(F90) $(F90FLAGS) -c global.f95

principal.o: precisao.mod principal.f95
	$(F90) $(F90FLAGS) -c principal.f95

integrando.o: precisao.mod integrando.f95
	$(F90) $(F90FLAGS) -c integrando.f95

integrais.o: precisao.mod integrais.f95
	$(F90) $(F90FLAGS) -c integrais.f95

# Limpando tudo
clean:
	rm precisao.mod test
	rm $(objects)

# Fim do makefile
```

Se alguém quiser remover a otimização ou compilar com diferentes opções de otimização, tudo o que ele precisa fazer é redefinir essa variável no início do makefile.

Como exemplo vamos gerar esse makefile e vamos chamá-lo de makefile2:

In [65]:
%%writefile exemplo_make/makefile2
# Esta é uma linha de comentário em um makefile
# Início do  makefile

objects = global.o integrais.o integrando.o principal.o
F90 = gfortran-8
F90FLAGS = -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008

# Makefile
test: $(objects)
	$(F90) -o test $(objects)

precisao.mod: global.o global.f95
	$(F90) $(F90FLAGS) -c global.f95

global.o: global.f95
	$(F90) $(F90FLAGS) -c global.f95

principal.o: precisao.mod principal.f95
	$(F90) $(F90FLAGS) -c principal.f95

integrando.o: precisao.mod integrando.f95
	$(F90) $(F90FLAGS) -c integrando.f95

integrais.o: precisao.mod integrais.f95
	$(F90) $(F90FLAGS) -c integrais.f95

# Limpando tudo
clean:
	rm precisao.mod test
	rm $(objects)

# Fim do makefile

Overwriting exemplo_make/makefile2


In [66]:
# Vamos executar o make
! cd exemplo_make && make -f makefile2

gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c global.f95
gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c global.f95
gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c integrais.f95
gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c integrando.f95
gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c principal.f95
gfortran-8 -o test global.o integrais.o integrando.o principal.o


In [67]:
# Vamos rodar ele
! echo '1 4\n 100\n' | ./exemplo_make/test

 Entre com os limites de integracao
 Entre com o numero ptos.
  Exato          :   -5.0626268927713607     
  Simpson13     :   -5.0626268941696519       Erro =    1.3982912605570164E-009
  Trapezio       :   -5.0624262873207657       Erro =    2.0060545059497059E-004



In [68]:
# Agora vamos limpar 
! cd exemplo_make && make -f makefile2 clean
! ls -alh exemplo_make/*

rm precisao.mod test
rm global.o integrais.o integrando.o principal.o
-rw-r--r-- 1 salviano salviano 2,9K fev  2 09:22 exemplo_make/global.f95
-rw-r--r-- 1 salviano salviano 3,0K fev  2 09:22 exemplo_make/integrais.f95
-rw-r--r-- 1 salviano salviano  292 fev  2 09:22 exemplo_make/integrando.f95
-rw-r--r-- 1 salviano salviano  682 fev  2 09:22 exemplo_make/Makefile
-rw-r--r-- 1 salviano salviano  670 fev  2 09:22 exemplo_make/makefile1
-rw-r--r-- 1 salviano salviano  750 fev  2 09:22 exemplo_make/makefile2
-rw-r--r-- 1 salviano salviano  678 fev  2 08:12 exemplo_make/makefile3
-rw-r--r-- 1 salviano salviano  845 fev  2 09:16 exemplo_make/makefile4
-rw-r--r-- 1 salviano salviano 2,5K fev  2 09:22 exemplo_make/principal.f95


## Regras de inferência 

As regras de inferência generalizam o processo de construção para que não seja necessário fornecer uma regra explícita para cada destino. Como exemplo, a compilação de fontes Fortran 90 (arquivos `.f95`) em arquivos objeto (arquivos `.o`) é uma ocorrência comum, como pode ser visto no exemplo acima. Em vez de exigir uma declaração de que cada arquivo `.o` depende de um arquivo `.f95` com o mesmo nome, use uma regra de inferência para inferir essa dependência. A fonte determinada por uma regra de inferência é chamada de fonte inferida. As regras de inferência são regras distinguidas pelo uso do caractere `%`  na linha de dependência/regra do makefile. Portanto, é possível fornecer um conjunto de regras para criar arquivos com um determinado sufixo a partir de arquivos com o mesmo nome de arquivo raiz (ou aproximadamente o mesmo), mas com um sufixo diferente. Por exemplo, as seguintes linhas no makefile:

```
    %.o: %.f95
            $(F90) -c $(F90OPT) $<

```

informa que todos os arquivos `.o` são criados a partir dos arquivos **.f95** correspondentes. O comando que se segue recompilará qualquer arquivo `.f95` se for mais recente que o arquivo `.o` correspondente. Portanto, essa linha usa a macro interna make `$<`, que significa "qualquer dependência que seja mais recente que seu destino correspondente". Essa macro interna só pode ser usada em regras de sufixo. Exceções à regra de sufixo podem ser declaradas explicitamente. 


Usando essa regra de sufixo recém-introduzida, o makefile acima descrito pode ser algo como isto:

Uma primeira versão 

```makefile
# Início do  makefile
objects = global.o integrais.o integrando.o principal.o
F90 = gfortran

F90FLAGS = -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008

# Makefile

test: $(objects)
	$(F90) -o test $(objects)

precisao.mod: global.o global.f95
	$(F90) $(F90FLAGS) -c global.f95

principal.o: precisao.mod principal.f95
	$(F90) $(F90FLAGS) -c principal.f95

integrando.o: precisao.mod integrando.f95
	$(F90) $(F90FLAGS) -c integrando.f95

integrais.o: precisao.mod integrais.f95
	$(F90) $(F90FLAGS) -c integrais.f95

%.o: %.f95
	$(F90) -c $(F90OPT) $<

# Limpando tudo
clean:
	rm precisao.mod test
	rm $(objects)
# Fim do makefile
```

In [69]:
%%writefile exemplo_make/makefile3
# Início do  makefile
objects = global.o integrais.o integrando.o principal.o
F90 = gfortran

F90FLAGS = -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008

# Makefile

test: $(objects)
	$(F90) -o test $(objects)

precisao.mod: global.o global.f95
	$(F90) $(F90FLAGS) -c global.f95

principal.o: precisao.mod principal.f95
	$(F90) $(F90FLAGS) -c principal.f95

integrando.o: precisao.mod integrando.f95
	$(F90) $(F90FLAGS) -c integrando.f95

integrais.o: precisao.mod integrais.f95
	$(F90) $(F90FLAGS) -c integrais.f95

%.o: %.f95
	$(F90) -c $(F90OPT) $<

# Limpando tudo
clean:
	rm precisao.mod test
	rm $(objects)
# Fim do makefile

Overwriting exemplo_make/makefile3


In [70]:
# Vamos executar o make
! cd exemplo_make && make -f makefile3

gfortran -c  global.f95
gfortran -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c global.f95
gfortran -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c integrais.f95
gfortran -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c integrando.f95
gfortran -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c principal.f95
gfortran -o test global.o integrais.o integrando.o principal.o


In [71]:
# Vamos rodar ele
! echo '1 4\n 100\n' | ./exemplo_make/test

 Entre com os limites de integracao
 Entre com o numero ptos.
  Exato          :   -5.0626268927713607     
  Simpson13     :   -5.0626268941696519       Erro =    1.3982912605570164E-009
  Trapezio       :   -5.0624262873207657       Erro =    2.0060545059497059E-004



In [72]:
# Agora vamos limpar 
! cd exemplo_make && make -f makefile3 clean
! ls -alh exemplo_make/*

rm precisao.mod test
rm global.o integrais.o integrando.o principal.o
-rw-r--r-- 1 salviano salviano 2,9K fev  2 09:22 exemplo_make/global.f95
-rw-r--r-- 1 salviano salviano 3,0K fev  2 09:22 exemplo_make/integrais.f95
-rw-r--r-- 1 salviano salviano  292 fev  2 09:22 exemplo_make/integrando.f95
-rw-r--r-- 1 salviano salviano  682 fev  2 09:22 exemplo_make/Makefile
-rw-r--r-- 1 salviano salviano  670 fev  2 09:22 exemplo_make/makefile1
-rw-r--r-- 1 salviano salviano  750 fev  2 09:22 exemplo_make/makefile2
-rw-r--r-- 1 salviano salviano  678 fev  2 09:22 exemplo_make/makefile3
-rw-r--r-- 1 salviano salviano  845 fev  2 09:16 exemplo_make/makefile4
-rw-r--r-- 1 salviano salviano 2,5K fev  2 09:22 exemplo_make/principal.f95


## Variáveis ou macros do GNU make


O aplicativo GNU make no auxilia lendo e interpretando um arquivo *makefile.* Além da capacidade básica de construir alvos de dependentes, o GNU make inclui muitos recursos que facilitam expressar as dependências e regras para construção de um alvo de seus dependentes.

Para compilar um grande número de arquivos Fortran usando o gfortran com as mesmas opções, por exemplo, digitar as opções para cada arquivo é tedioso. Pode-se evitar essa tarefa repetitiva definindo uma variável ou macro no make da seguinte maneira:


```makefile
# Define um macro para o nome do compilador
FC= gfortran
# Defina um macro para cada FLAG de compilação do gfortran
FFLAGS= -O3 -Wall -std=f2008
# Defina a regra para a construção de um arquivo objeto
rotina.o: rotina.f95 modulo.mod
$(FC) -c $(FFLAGS) rotina.f95
```

Neste exemplo, `FC` e `FFLAGS` são variáveis do make. (O GNU make prefere chamá-los de *variables,* mas a maioria dos usuários Unix os chamam de *macros*.)

Para usar uma variável em qualquer lugar no makefile, inicie com o sinal (`$`) seguido pela variável entre parênteses. O GNU make substitui todas as ocorrências de uma variável por sua definição; assim, ele substitui todas as ocorrências de `$(FFLAGS)` pela string `-O3 -Wall -std=f2008`.

O GNU make possui várias variáveis predefinidas que possuem significados especiais. Esta tabela lista essas variáveis. Além das variáveis listadas aqui, o GNU make considera que todas as variáveis de ambiente (como PATH e HOME) também são variáveis predefinidas.


| Variável   | Significado                                                  |
| :---------- | :------------------------------------------------------------ |
| `$@`       | Nome completo do alvo. Se o alvo é libDisp.a (image.o), por exemplo, `$@` é libDisp.a. |
| `$%`       | Nome do membro para destinos que são arquivos. Se o alvo for libDisp.a (image.o), por exemplo, `$%` é image.o. |
| `$*`       | Nome do arquivo alvo sem a extensão.                         |
| `$+`       | Nomes de todos os arquivos dependentes com dependências duplicadas, listados em sua ordem de ocorrência. |
| `$<`       | O nome do primeiro arquivo dependente.                       |
| `$?`       | Nomes de todos os arquivos dependentes (com espaços entre os nomes) que são mais recentes que o destino. |
| `$^`       | Nomes de todos os arquivos dependentes, com espaços entre os nomes. Duplicatas são removidas dos nomes de arquivos dependentes. |
| `AR`       | Nome do programa de manutenção de archive (valor padrão: `ar`). |
| `ARFLAGS`  | Flags para o programa de manutenção de arquivo (valor padrão: `rv`). |
| `AS`       | Nome do programa assembler que converte a linguagem de montagem em código de objeto (valor padrão: `as`). |
| `ASFLAGS`  | Flags para o montador.                                       |
| `CC`       | Nome do compilador C (valor padrão: `cc`).                   |
| `CFLAGS`   | Flags que são  passadas ao compilador C.                     |
| `CO`       | Nome do programa que extrai um arquivo do RCS (valor padrão: `co`). |
| `COFLAGS`  | Flags para o programa RCS `co`.                              |
| `CPP`      | Nome do pré-processador C (valor padrão: `$ (CC) -E`).       |
| `CPPFLAGS` | Flags para o preprocessador  C .                             |
| `CXX`      | Nome do compilador C++  (valor padrão: `g++`).               |
| `CXXFLAGS` | Flags que são  passadas ao compilador C++.                   |
| `FC`       | Nome do compilador FORTRAN  (valor padrão: `f77`).           |
| `FFLAGS`   | Flags que são  passadas ao compilador FORTRAN                |
| `LDFLAGS`  | Sinalizadores para o compilador quando é necessário invocar o linker `ld`. |
| `RM`       | Nome do comando para deletar um arquivo (valor padrão: `rm -f`). |


A tabela acima, mostrou algumas das variáveis predefinidas mais comumente usadas. Esta lista não é comleta e os valores padrão mostrados aqui podem não ser o que faz as seleções para o seu ambiente. Consulte o manual do Gnu make a [tabela de variáveis automáticas](https://www.gnu.org/software/make/manual/make.html#Variables-Simplify).


Para ver a lista completa de variáveis predefinidas para sua instância do GNU, é possível executar `make -p` em um diretório sem makefiles.

```bash
make -p
```

Um outro exemplo, um pouco mais elaborado definindo as variáveis padrões em códigos Fortran 90/95 mais elaborados.


```makefile
# Início do  makefile

#Na linha abaixo defini-se o nome do executável
PROG =	test

SRCS =	global.f95 integrais.f95 integrando.f95 principal.f95

OBJS =	global.o integrais.o integrando.o principal.o

# Aqui pode-se definir as bibliotecas, por exemplo, a lapack
LIBS =

F90 = gfortran-8

F90FLAGS = -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008

# Opções para link
LDFLAGS = -s

# Alvo padrão: all
all: $(PROG)

$(PROG): $(OBJS)
	$(F90) $(LDFLAGS) -o $@ $(OBJS) $(LIBS)


# Limpando tudo
clean:
	rm -f $(PROG) $(OBJS) *.mod

.SUFFIXES: $(SUFFIXES) .f95

.f95.o:
	$(F90) $(F90FLAGS) -c $<

# Fim do makefile
```

O exemplo acima, funciona acidentalmente, porque a lista de arquivos definidas em `OBJS` esta na ordem de dependência correta. Se colocarmos `global.o` como sendo o últimos da lista ele seria o último a ser gerado, entretanto todos os outros dependem dele. Falta portanto escrever as dependências. Essa, para grandes códigos é a tarefa mais difícil. Entretano há um aplicativo linux que resolve esse problema, ele é o `makedepf90`, nas distribuições baseadas no Debian, ele pode ser instalado com o seguinte comando:

```bash
sudo apt install makedepf90
```

Nesse caso para gerar as dependências corretas faça

In [73]:
! cd exemplo_make && makedepf90 *.f95

global.o : global.f95 
integrais.o : integrais.f95 global.o 
integrando.o : integrando.f95 global.o 
principal.o : principal.f95 global.o 


Ao inserir as dependências no makefile obtemos o seguinte arquivo:

In [74]:
%%writefile exemplo_make/makefile4
# Início do  makefile

#Na linha abaixo defini-se o nome do executável
PROG =	test

SRCS =	integrais.f95 integrando.f95 principal.f95 global.f95

OBJS =	integrais.o integrando.o principal.o global.o 

# Aqui pode-se definir as bibliotecas, por exemplo, a lapack
LIBS =

F90 = gfortran-8

F90FLAGS = -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008

# Opções para link
LDFLAGS = -s

# Alvo padrão: all
all: $(PROG)

$(PROG): $(OBJS)
	$(F90) $(LDFLAGS) -o $@ $(OBJS) $(LIBS)


# Limpando tudo
clean:
	rm -f $(PROG) $(OBJS) *.mod

.SUFFIXES: $(SUFFIXES) .f95

.f95.o:
	$(F90) $(F90FLAGS) -c $<
    
# As dependencias geradas pelo makedepf90
global.o : global.f95 
integrais.o : integrais.f95 global.o 
integrando.o : integrando.f95 global.o 
principal.o : principal.f95 global.o

# Fim do makefile

Overwriting exemplo_make/makefile4


In [75]:
# Vamos executar o make
! cd exemplo_make && make -f makefile4

gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c global.f95
gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c integrais.f95
gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c integrando.f95
gfortran-8 -O3 -Wall -Waliasing -pedantic -Warray-bounds -Wsurprising -Wunderflow -fcheck=all -std=f2008 -c principal.f95
gfortran-8 -s -o test integrais.o integrando.o principal.o global.o  


In [76]:
# Vamos rodar ele
! echo '1 4\n 100\n' | ./exemplo_make/test

 Entre com os limites de integracao
 Entre com o numero ptos.
  Exato          :   -5.0626268927713607     
  Simpson13     :   -5.0626268941696519       Erro =    1.3982912605570164E-009
  Trapezio       :   -5.0624262873207657       Erro =    2.0060545059497059E-004



In [77]:
# Agora vamos limpar 
! cd exemplo_make && make -f makefile4 clean
! ls -alh exemplo_make/*

rm -f test integrais.o integrando.o principal.o global.o  *.mod
-rw-r--r-- 1 salviano salviano 2,9K fev  2 09:22 exemplo_make/global.f95
-rw-r--r-- 1 salviano salviano 3,0K fev  2 09:22 exemplo_make/integrais.f95
-rw-r--r-- 1 salviano salviano  292 fev  2 09:22 exemplo_make/integrando.f95
-rw-r--r-- 1 salviano salviano  682 fev  2 09:22 exemplo_make/Makefile
-rw-r--r-- 1 salviano salviano  670 fev  2 09:22 exemplo_make/makefile1
-rw-r--r-- 1 salviano salviano  750 fev  2 09:22 exemplo_make/makefile2
-rw-r--r-- 1 salviano salviano  678 fev  2 09:22 exemplo_make/makefile3
-rw-r--r-- 1 salviano salviano  845 fev  2 09:22 exemplo_make/makefile4
-rw-r--r-- 1 salviano salviano 2,5K fev  2 09:22 exemplo_make/principal.f95


Não há muita necessidade para esta regra de sufixo no exemplo acima, devido à sua simplicidade. Mas, por essa regra, todo arquivo de código-fonte (no espaço de trabalho do projeto) que não usa o arquivo de módulo será incluído no makefile. Desta forma, makefiles para projetos muito grandes (com grande número de arquivos de código-fonte) podem ser facilmente gerados/escritos. Dessa forma, usando regras de sufixo, não é necessário escrever a regra de makefile para cada arquivo de código-fonte. Uma regra de sufixo com `$<` definirá a regra makefile para a maioria dos arquivos de código-fonte. Regras adicionais para os arquivos de código-fonte que usam módulos são gravadas separadamente. Regra que afirma que o arquivo de módulo precisa ser compilado antes que o arquivo de código-fonte (que usa esse arquivo de módulo) ainda seja válido.

## Pesquisando Diretórios para Pré-requisitos

Para sistemas grandes, muitas vezes é desejável colocar fontes em um diretório separado dos binários. Os recursos de pesquisa de diretório do make facilitam isso pesquisando vários diretórios automaticamente para encontrar um pré-requisito. Quando se redistribui os arquivos entre diretórios, não é necessário alterar as regras individuais, apenas os caminhos de pesquisa.


O valor da variável `VPATH` do make especifica uma lista de diretórios que devem ser pesquisados. Na maioria das vezes, espera-se que os diretórios contenham arquivos de pré-requisito que não estão no diretório atual; no entanto, use `VPATH` como uma lista de pesquisa para os pré-requisitos e os destinos das regras.

Portanto, se um arquivo listado como destino ou pré-requisito não existir no diretório atual, faça as pesquisas nos diretórios listados no `VPATH` para um arquivo com esse nome. Se um arquivo for encontrado em um deles, esse arquivo pode se tornar o pré-requisito (veja abaixo). As regras podem, então, especificar os nomes dos arquivos na lista de pré-requisitos, como se todos eles existissem no diretório atual. 

Na variável `VPATH`, os nomes dos diretórios são separados por dois pontos ou espaços em branco. A ordem em que os diretórios são listados é a ordem seguida por make em sua pesquisa. (No MS-DOS e no MS-Windows, os pontos e vírgulas são usados ​​como separadores de nomes de diretórios no VPATH, já que os dois pontos podem ser usados no próprio nome do caminho, depois da letra da unidade.)

Por exemplo,

```makefile
VPATH = src: ../modulos
```

especifica um caminho contendo dois diretórios, `src` e `../modulos`, que fazem pesquisas nessa ordem.

Com esse valor de `VPATH`, a seguinte regra,

```
foo.o: foo.c
```

é interpretado como se estivesse escrito assim:

```
foo.o: src/foo.c
```

assumindo que o arquivo foo.c não existe no diretório atual, mas é encontrado no diretório src.

## Comentários finais

Este foi um tutorial introdutório muito breve para criar makefiles simples, que poderiam ser usados na programação diária, a fim de acelerar o processo de construção de programas fortran no ambiente Linux. Makefiles são ferramentas muito úteis para construir grandes programas consistindo em dezenas de arquivos de código fonte (incluindo arquivos de módulo). Tudo que foi dito aqui implicou que se estava usando o Fortran como uma linguagem de programação. Outro tutorial simples para criar makefiles para a linguagem Fortran pode ser encontrado [aqui](http://www.nersc.gov/nusers/help/tutorials/make/print.php). Também é possível criar makefiles semelhantes para a linguagem de programação C. Um tutorial mais complexo para criar makefiles pode ser encontrado [aqui](http://users.actcom.co.il/~choo/lupg/tutorials/writing-makefiles/writing-makefiles.html) (está escrito para a linguagem C).